### References
##### Physics-informed neural networks: A deep learning framework for solving forward and inverse problems involving nonlinear partial differential equations 

https://www.sciencedirect.com/science/article/pii/S0021999118307125

##### Physics Informed Deep Learning (Part I): Data-driven Solutions of Nonlinear Partial Differential Equations

https://arxiv.org/pdf/1711.10561.pdf

##### Physics Informed Deep Learning (Part II): Data-driven Discovery of Nonlinear Partial Differential Equations

https://arxiv.org/pdf/1711.10566.pdf

##### Authors: Maziar Raissi, Paris Perdikaris, George Em Karniadakis

https://github.com/maziarraissi/PINNs <br>
https://maziarraissi.github.io/PINNs/

### 1)Burgers' equation
Physics form:
$$\frac{\partial u(t,x)}{\partial t} + u(t,x) \frac{\partial u(t,x)}{\partial x} = \nu \frac{\partial^2 u(t,x)}{\partial x^2}$$
$u(t,x)$: velocity of fluid, $\nu$: viscosity of fluid <br><br>

General form:
$$\frac{\partial u(t,x)}{\partial t} + \lambda _{1} u(t,x) \frac{\partial u(t,x)}{\partial x} -\lambda _{2} \frac{\partial^2 u(t,x)}{\partial x^2} = 0$$<br>

In Maziar Raissi, Paris Perdikaris, and George Em Karniadakis paper: <br>
$$\frac{\partial u(t,x)}{\partial t} + u(t,x) \frac{\partial u(t,x)}{\partial x} -\frac{0.01}{\pi} \frac{\partial^2 u(t,x)}{\partial x^2} = 0$$ <br>
$$u(0,x) = -sin(\pi x) \textrm{, which is the initial condition}$$ <br>
$$u(t,-1) = u(t,1) = 0 \textrm{, which is the Dirichlet boundary conditions}$$<br>
$$x \in [-1,1], \textrm{ }t\in [0,1]$$
Close to the exact solution is $u(t,x) = e^{-t}sin(\pi x)$, [here is the exact analytical solution.](https://www.sciencedirect.com/science/article/abs/pii/0045793086900368)

### 2)PINN approach: Inference => find u(t,x) (Appendix A1 Data-driven solution of partial differential equations)
In this part, we are looking to find the exact solution of the PDE (partial derivate equation), $u(t,x)$. The PDE, boundary and inital conditions are known, $u(t,x)$ is unknown.

Define $f := u_t + uu_x - \frac{0.01}{\pi}u_{xx}$, this function is an approximation of the Burgers' equation / the true PDE given by the PINN. <br>

```python
def f(t,x):
    u=u(t,x)
    u_t=tf.gradients(u,t)[0]
    u_x=tf.gradients(u,x)[0]
    u_xx=tf.gradients(u_x,x)[0]
    f=u_t+u∗u_x−(0.01/tf.pi)∗u_xx
return f
```

```u(t,x)``` is the approximation given by the PINN of the solution that satisfies the true PDE. ```u(t,x)``` PINN approximation of $u(t,x)$ the exact solution.

```python
def u(t,x):
    u=neural_net(tf.concat([t,x],1),weights,biases)
return u
```

In other words: ```u(t,x)= PINN(t,x) == PINN u``` and ```f(t,x) == PINN PDE``` <br><br>

The loss is defined by: $MSE = MSE_f + MSE_u$, it is a combination of the PDE loss $MSE_f$ and the boundary & initial conditions loss $MSE_u$. <br>
$MSE_f = \frac{1}{N_f}\Sigma_{i=1}^{N_f} |f(t_f^i,x_f^i)|^2$, where $\{t_f^i,x_f^i\}_{i=1}^{N_f}$ is the grid generated for the PDE. <br>
$MSE_u = \frac{1}{N_u}\Sigma_{i=1}^{N_u} |u(t_u^i,x_u^i) - u^i|^2$, where $\{t_u^i,x_u^i\}_{i=1}^{N_u}$ is the grid generated for the boundary & initial conditions, ```u^i``` is an exact solution point from the boundary and initial conditions, ```u(t_u^i,x_u^i)``` is an approximation of $u(t,x)$ (the exact solution) given by the PINN.<br><br>

Architecture used:
- 9 layers
- 20 neurons
- LBFGS optimizer
- Weights xavier init
- Activation function: tanh
- Nu = 100: bc and ic points
- Nf = 10000: PDE / collocation points

Remark: we want to solve the true PDE without knowing its solution, it can be seen as an unsupervised task where the data required is only the boundary and initial conditions (t = 0, x = -1, 1)).

**POURQUOI NF EST PAS SUR LE PLOT, en interne dans le NN?**

<div>
<img src="1.PNG" width="500"/>
</div>
<div>
<img src="2.PNG" width="500"/>
</div>

### 3)PINN Approach: Identification => find the PDE (Appendix B1 Data-driven discovery of partial differential equations)

In this part, the solution $u(x,t)$ is known and we want to discover the parameters of the equation $F(t, x, u(t, x), \lambda) = 0.$ This is a supervised task, the dataset is constituted of $N$ points, which are randomly selected in the entire space-time domain  $\Omega = [0, 1] \times [-1, 1]$, $S = \{t_i, x_i, u(t_i, x_i)\}_{i=1}^N$, this dataset gives the solution for every $(t, x) \in \Omega$. The information from the boundary & intial conditions is encoded in the dataset, $x \in [-1,1]$, $t \in [0,1]$ and $u(t,x)$ in the interior of $\Omega$. <br>

$u(t,x)$ the exact solution is known, the true PDE is unknown, the boundary and inital conditions are known and are only used for the generation of points in $\Omega$. <br>

The general Burgers' equation in 1D is: $$\frac{\partial u(t,x)}{\partial t} + \lambda _{1} u(t,x) \frac{\partial u(t,x)}{\partial x} -\lambda _{2} \frac{\partial^2 u(t,x)}{\partial x^2} = 0$$ 

Define $f := u_t + \lambda _1 u u_x - \lambda _2 u_{xx}$, (link with $F$: $\lambda = (\lambda_1,\lambda_2)$), this will be an approximation of the PDE given by the PINN, ```u(t,x)``` approximation of the exact solution given by the PINN.

The loss is defined by: $MSE = MSE_f + MSE_u$, it is a combination of the PDE loss $MSE_f$ and the training data loss $MSE_u$. <br>
$MSE_f = \frac{1}{N}\Sigma_{i=1}^{N} |f(t_u^i,x_u^i)|^2$ <br>
$MSE_u = \frac{1}{N}\Sigma_{i=1}^{N} |u(t_u^i,x_u^i) - u^i|^2$ <br>

Where $\{t_u^i,x_u^i,u^i\}_{i=1}^{N}$ is the training dataset.

Architecture used:
- 9 layers
- 20 neurons
- LBFGS optimizer
- Weights xavier init
- Activation function: tanh
- N = 2000 

Expected $\lambda$: $\lambda_1 = 1.0$ and $\lambda_2 = \frac{0.01}{\pi} \approx 0.00318$

<div>
<img src="3.PNG" width="500"/>
</div>
<div>
<img src="4.PNG" width="500"/>
</div>

### 4)Implementation of Burgers' equation Inference in Pytorch 